In [1]:
#import needed librarys
from bs4 import BeautifulSoup
import requests
import time, os
from selenium.webdriver.common.keys import Keys
from functools import reduce, partial
#libraries for data and modeling
import numpy as np
import pandas as pd


In [59]:
#modified webdriver code for firefox
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
options.log.level = "trace"
#options.add_argument("--headless") #option to not pull up and show the web browser

driver = webdriver.Firefox(options=options, executable_path='/usr/local/bin/geckodriver')

In [43]:
#function to randomize sleep times to help spoof activity
def human_imposter(n = 60):
    '''
    This function randomly selects a wait time (s)
    to aid impersonating a person on a website. 
    The high end for random selection is n with
    default value 60 seconds.
    '''
    i = round(np.random.uniform(low=15,high=n),3)
    time.sleep(i)

In [60]:
#go to main login in page
url = 'https://www.formula1.com/en/results.html/2021/races/1064/bahrain/race-result.html' 
driver.get(url)
print('You have 1 minute to manually pass the cookies popup')
time.sleep(60)

You have 1 minute to manually pass the cookies popup


In [27]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [28]:
str_records = ["_".join(i.text.split()) for i in soup.find_all('a',{'data-name':'resultType'})]
str_records

['Race_result',
 'Fastest_laps',
 'Pit_stop_summary',
 'Starting_grid',
 'Qualifying',
 'Practice_3',
 'Practice_2',
 'Practice_1']

In [29]:
str_locations = [i.findNext().text for i in soup.find_all('a',{'data-name':'meetingKey'})[1:]]
str_locations

['Bahrain',
 'Emilia Romagna',
 'Portugal',
 'Spain',
 'Monaco',
 'Azerbaijan',
 'France',
 'Styria',
 'Austria',
 'Great Britain',
 'Hungary',
 'Belgium',
 'Netherlands',
 'Italy',
 'Russia',
 'Turkey',
 'United States ',
 'Mexico',
 'Brazil',
 'TBC',
 'Saudi Arabia',
 'Abu Dhabi']

In [30]:
str_years = [i.findNext().text for i in soup.find_all('a',{'data-name':'year'})]
str_years[:16]

['2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006']

In [232]:
years = driver.find_elements_by_xpath('//a[@data-name="year"]')

In [233]:
locations = driver.find_elements_by_xpath('//a[@data-name="meetingKey"]')

In [234]:
records = driver.find_elements_by_xpath('//a[@data-name="resultType"]')

In [9]:
def get_table_data(record):    
    temp_soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = temp_soup.find('table')
    rows = [row for row in table.find_all('tr')] 
    my_list = []
    headers = ['_'.join(i.text.strip().split()) for i in rows[0].find_all('th')[1:-1]]
    headers = [(record+'_'+header)  if header!='Driver' else header for header in headers]
    for row in rows[1:]:
        temp_row = [ ' '.join(j.text.strip().split()) for j in row.find_all('td')[1:-1]]  
        my_list.append(dict(zip(headers,temp_row)))

    return(pd.DataFrame(my_list))

In [40]:
def get_records_df(location):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    if (str_records != ["_".join(i.text.split()) for i in soup.find_all('a',{'data-name':'resultType'})]):
        return None
    my_dict = {}
    for i, record in enumerate(str_records):
        records = driver.find_elements_by_xpath('//a[@data-name="resultType"]')
        records[i].click()
        human_imposter(15) #need time for loading tables
        my_dict[record] = get_table_data(record)

    my_reduce = partial(pd.merge, on='Driver', how='outer') 
    temp = reduce(my_reduce, my_dict.values())
    temp['location'] = location
    return temp

In [41]:
def get_loc_df(year):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    str_locations = [i.findNext().text for i in soup.find_all('a',{'data-name':'meetingKey'})[1:]]
    my_dict = {}
    for i, location in enumerate(str_locations):
        locations = driver.find_elements_by_xpath('//a[@data-name="meetingKey"]')
        driver.execute_script("window.scrollTo(0, 0);"); #can't click unless you scroll up all the way
        time.sleep(3) #allow time for scroll
        locations[i+1].click() #need +1 to avoid All category
        human_imposter(15)
        temp_df = get_records_df(location)
        if temp_df is None:
            continue #iterate past races without standard records
        my_dict[location] = temp_df
    temp = pd.concat(my_dict.values(),ignore_index = True)
    temp['year'] = year
    return temp

In [61]:
my_dict = {}
for i, year in enumerate(str_years[9:16]):
    years = driver.find_elements_by_xpath('//a[@data-name="year"]')
    driver.execute_script("window.scrollTo(0, 0);"); #can't click unless you scroll up all the way
    time.sleep(3) #allow time for scroll
    years[i+9].click()
    human_imposter(15)
    my_dict[year] = get_loc_df(year)
df = pd.concat(my_dict.values(),ignore_index = True)
df

,Race_result_Pos,Race_result_No,Driver,Race_result_Car,Race_result_Laps,Race_result_Time/Retired,Race_result_PTS,Fastest_laps_Pos,Fastest_laps_No,Fastest_laps_Car,...,Practice_2_Gap,Practice_2_Laps,Practice_1_Pos,Practice_1_No,Practice_1_Car,Practice_1_Time,Practice_1_Gap,Practice_1_Laps,location,year
0,1,3,Jenson Button BUT,McLaren Mercedes,58,1:34:09.565,25,1,3,McLaren Mercedes,...,+3.856s,18,1,3,McLaren Mercedes,1:27.560,,11,Australia,2012
1,1,3,Jenson Button BUT,McLaren Mercedes,58,1:34:09.565,25,1,3,McLaren Mercedes,...,+3.856s,18,1,3,McLaren Mercedes,1:27.560,,11,Australia,2012
2,2,1,Sebastian Vettel VET,Red Bull Racing Renault,58,+2.139s,18,2,1,Red Bull Racing Renault,...,+3.011s,19,11,1,Red Bull Racing Renault,1:29.790,+2.230s,21,Australia,2012
3,2,1,Sebastian Vettel VET,Red Bull Racing Renault,58,+2.139s,18,2,1,Red Bull Racing Renault,...,+3.011s,19,11,1,Red Bull Racing Renault,1:29.790,+2.230s,21,Australia,2012
4,3,4,Lewis Hamilton HAM,McLaren Mercedes,58,+4.075s,15,4,4,McLaren Mercedes,...,+4.069s,11,2,4,McLaren Mercedes,1:27.805,+0.245s,14,Australia,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,NaN,NaN,Sebastian Vettel VET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,+0.323s,33,4,38,Sauber BMW,1:14.204,+0.440s,29,Brazil,2006
5767,NaN,NaN,Franck Montagny MON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,+1.245s,20,17,41,Super Aguri Honda,1:17.744,+3.980s,6,Brazil,2006
5768,NaN,NaN,Michael Ammermuller AMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,+1.889s,31,10,37,RBR Ferrari,1:15.711,+1.947s,22,Brazil,2006
5769,NaN,NaN,Ernesto Viso VIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,+2.425s,26,13,39,MF1 Toyota,1:16.737,+2.973s,32,Brazil,2006


In [56]:
df.columns

Index(['Race_result_Pos', 'Race_result_No', 'Driver', 'Race_result_Car',
       'Race_result_Laps', 'Race_result_Time/Retired', 'Race_result_PTS',
       'Fastest_laps_Pos', 'Fastest_laps_No', 'Fastest_laps_Car',
       'Fastest_laps_Lap', 'Fastest_laps_Time_of_day', 'Fastest_laps_Time',
       'Fastest_laps_Avg_Speed', 'Pit_stop_summary_Stops',
       'Pit_stop_summary_No', 'Pit_stop_summary_Car', 'Pit_stop_summary_Lap',
       'Pit_stop_summary_Time_of_day', 'Pit_stop_summary_Time',
       'Pit_stop_summary_Total', 'Starting_grid_Pos', 'Starting_grid_No',
       'Starting_grid_Car', 'Starting_grid_Time', 'Qualifying_Pos',
       'Qualifying_No', 'Qualifying_Car', 'Qualifying_Q1', 'Qualifying_Q2',
       'Qualifying_Q3', 'Qualifying_Laps', 'Practice_3_Pos', 'Practice_3_No',
       'Practice_3_Car', 'Practice_3_Time', 'Practice_3_Gap',
       'Practice_3_Laps', 'Practice_2_Pos', 'Practice_2_No', 'Practice_2_Car',
       'Practice_2_Time', 'Practice_2_Gap', 'Practice_2_Laps',
       'Pr

In [62]:
df.to_csv('2012_2006.csv')

In [63]:
driver.close()